# *학습 관련 기술들*

## 1.  매개변수 갱신

> - Optimization(최적화)은 손실함수의 최소값을 갖는 매개변수를 찾는 문제
- 고차원 신경망에서 이를 찾기란 쉽지 않지만 반복적으로 매개변수의 기울기(미분)를 찾아가며 근사할 수 있음
-  매개변수를 갱신하는 최적화 기법에는 SGD, Momemtum, AdaGrad, Adam 4종류가 있음

### 1) SGD (확률적 경사 하강법)

- SGD는 수식으로 다음과 같이 표현 가능
$$
W ← W - \eta\frac{\delta L}{\delta W}
$$
- $W$는 갱신할 매개변수고, $\frac{\delta L}{\delta W}$는 손실함수의 기울기로 여깅에 $\eta$만큼 값을 갱신한다는 의미
- SGD는 아래와 같이 구현 가능

In [1]:
class SGD:
    def __init__(self, lr = 0.01):
        self.lr = lr
       
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr*grads[key]

- 핵심이 되는 params[key] -= self.lr\*grads[key]는 params[key] = params[key] - self.lr\*grads[key]와 같은 공식임
- 해석하자면 매개편수인 params를 기울기에 학습률을 곱한만큼 반대 방향으로 이동기킨다고 볼 수 있음
- 실제로 사용할 때에는 아래와 같이 사용 가능

In [ ]:
network = TwoLayerNet(...)
optimizer = SGD()

for i in range(10000):
    ...
    x_batch, t_batch = get_mini_batch(...)
    grads = network.gradient(x_batch, t_batch)
    params = network.params
    optimizer.update(params, grads)
    ...

- 위와 같이 optimizer가 최적화를 수행할 수 있도록 '매개변수'와 '기울기' 정보만 제공하면 계산하도록 클래스 구현 ...(params, grads)
___

### 2) SGD의 단점

- SGD는 문제에 따라서 비효율적일 때가 많음
- 실제로 아래의 예제의 경우 최소값을 찾기까지 오랜 시행착오를 겪게 됨
$$
f(x,y) = \frac{1}{20}x^2+y^2
$$
![](image/fig 6-1.png)

- 위의 함수에 초기값 (-7, 2)를 입력하였을 경우 함수의 이동은 다음과 같음
![](image/fig 6-3.png)

- 이는 SGD의 단점으로 비등방성 함수 (방향에 따라 성질/기울기가 달라지는 함수) 에서는 탐색 경로가 비효율적이게 됨
___

### 3) 모멘텀

- **Momemtum**(모멘텀)은 운동량을 뜻하는 단어로, 수식으로 표현 시 다음과 같이 표현 가능

$$ 
v ← a v - \eta \frac{\delta L}{\delta W}
$$
$$
W ← W + v
$$

- 앞서 SGD 처럼 $W$는 매개변수, $\eta$는 학습률, $\frac{\delta L}{\delta W}$는 손실함수의 기울기를 의미
- $v$는 물리에서 말하는 속도(velocity)이며 $av$항은(위의 식에서) 물체가 아무런 힘을 받지 않을 때 서서히 하강시키는 역할을 함
- 모멘텀을 구현하면 아래와 같음

In [5]:
class Momemtun:
    def __init__(self, lr = 0.01, momentum = 0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
        
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_list(val)
                
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

- 3번째 문단이(for문) 중요한데, $v$에 모멘텀을 곱한 후 손실함수의 기울기를 더하여 학습률을 보정
- 실례를 보면 다음 같이 최적화를 찾아가는 것으로 볼 수 있음
![](image/fig 6-5.png)

- 이는 $x$축의 힘은 아주 작은 대신 방향변화가 없어 한 방향으로 일정하게 가속하기 때문
- 반대로 $y$축 방향의 속도는 위아래로 번갈아가며 속도가 안정적이지 않음
___

### 4) AdaGrad

- AdaGrad는 학습률을 조정하는 최적화 기법으로, 학습을 진행하면서 학습률을 점차 줄여가는 방법임
- 단, 매개변수 '전체'의 학습률을 일괄적으로 낮추는 대신, '각각의' 매개변수에 '맞춤형' 값을 반환
- 수식으로 표현하면 다음과 같음

$$
h ← h + \frac{\delta L}{\delta W}\odot\frac{\delta L}{\delta W}
$$
$$
W ← W - \eta\frac{1}{\sqrt h}\frac{\delta L}{\delta W}
$$

- 기존 notation 외, 새로 $h$라는 변수가 등장하며, $h$는 기존 기울기 값을 제곱하여 계속 더해 줌
- 그리고 매개변수 $W$를 갱신할 때 $\frac{1}{\sqrt h}$를 곱해주는데, 이를 해석하자면 **매개변수 중 많이 움직인 원소는 학습률이 낮아진다고** 할 수 있음

※ AdaGrad의 경우 과거의 기울기를 계속 제곱하여 더해가다 보니 어느순간 갱신량이 0에 근사하여 전혀 갱신되지 않음. 이를 개선한 기법이 RMSProp으로 먼 과거의 기울기는 서서히 잊고, 새로운 기울기 정보를 크게 반영함. 이를 지수이동평균(Exponential Moving Average)라고 함

- AdaGrad를 구현하면 아래와 같음

In [8]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.h[key] += grads[key]*grads[key]
            params[key] -= self.lr*grads[key]/(np.sqrt(self.h[key])+1e7)

- 제일 하단의 $\frac{1}{\sqrt h}$를 $\frac{1}{\sqrt h + 1e7}$으로 수정해 0으로 나누는 사례 방지
- AdaGrad를 사용해서 최적화 진행 시 아래와 같은 결과 확인 가능
![](image/fig 6-6.png)

- y축 방향은 기울기가 커서 처음에는 크게 움직이지만, 갱신 정도도 큰 폭으로 작아지도록 조정
- 따라서 y축 방향으로 갱신강도가 빠르게 약해지고 지그재그 움직임이 줄어듬
___

### 5) Adam

- Adam은 이론적으로는 복잡하지만 직관적으로 보면 그 원리가 Momemtum과 AdaGrad를 융합한 모습을 보임
- 그림으로 설명하자면 아래와 같이 표현 가능
![](image/fig 6-7.png)
___

### 6) MNIST 데이터 셋으로 본 갱신방법 비교

- 손글씨 숫자 인식을 대상으로 위의 네 기법을 비교해 보면 다음과 같은 결과 기대 가능
![](image/fig 6-9.png)
- 일반적으로 SGD의 학습 진도가 가장 느리고, 나머지 세 기법의 진도는 비슷